In [ ]:
# Running SQL in Python: Cheat Sheet
import sqlite3
import pandas as pd
from sqlalchemy import create_engine

# Setting Up SQLite Database (In-Memory or File-Based)
conn = sqlite3.connect(":memory:")  # Temporary DB (in RAM)
# conn = sqlite3.connect("database.db")  # Persistent DB
cursor = conn.cursor()

# Creating a Table
cursor.execute("""
CREATE TABLE users (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    age INTEGER,
    city TEXT
)
""")
conn.commit()

# Inserting Data
cursor.execute("INSERT INTO users (name, age, city) VALUES ('Alice', 25, 'New York')")
cursor.executemany("INSERT INTO users (name, age, city) VALUES (?, ?, ?)", [
    ("Bob", 30, "Chicago"),
    ("Charlie", 35, "San Francisco")
])
conn.commit()

# Querying Data
cursor.execute("SELECT * FROM users")  # Select all rows
rows = cursor.fetchall()  # Fetch results
for row in rows:
    print(row)

# Querying with Parameters (to prevent SQL injection)
age_limit = 30
cursor.execute("SELECT * FROM users WHERE age > ?", (age_limit,))
print(cursor.fetchall())

# Using Pandas with SQL
df = pd.read_sql_query("SELECT * FROM users", conn)  # Load SQL query into DataFrame
print(df)

# SQLAlchemy for Advanced Queries & Connection Handling
engine = create_engine("sqlite:///:memory:", echo=False)  # Create engine
df.to_sql("users", engine, index=False, if_exists="replace")  # Save DataFrame to SQL
df2 = pd.read_sql("SELECT * FROM users", engine)  # Load from SQL
print(df2)

# Updating & Deleting Data
cursor.execute("UPDATE users SET age = age + 1 WHERE name = 'Alice'")
cursor.execute("DELETE FROM users WHERE name = 'Bob'")
conn.commit()

# Joining Tables
cursor.execute("CREATE TABLE orders (id INTEGER PRIMARY KEY, user_id INTEGER, amount REAL)")
cursor.executemany("INSERT INTO orders (user_id, amount) VALUES (?, ?)", [
    (1, 100.50), (2, 200.75), (3, 50.25)
])
conn.commit()

query = """
SELECT users.name, orders.amount 
FROM users 
JOIN orders ON users.id = orders.user_id
"""
df_join = pd.read_sql_query(query, conn)
print(df_join)

# Transactions (Ensuring Atomicity)
try:
    cursor.execute("INSERT INTO users (name, age, city) VALUES ('Dave', 40, 'Boston')")
    cursor.execute("INSERT INTO users (name, age, city) VALUES ('Eve', 'invalid_age', 'LA')")  # Error!
    conn.commit()  # This will not execute due to the error
except Exception as e:
    conn.rollback()  # Roll back changes if an error occurs
    print("Transaction failed:", e)

# Closing Connection
cursor.close()
conn.close()
